# Conjunto de Dados 2: _China Set - The Shenzhen set - Chest X-ray Database_
***
> Disponível em: <https://www.kaggle.com/kmader/pulmonary-chest-xray-abnormalities>. Acesso em 19 fev. 2021.

### Importação dos pacotes necessários

In [2]:
import glob
import re
from tqdm import tqdm
import urllib.request
import pandas as pd
from sklearn.utils import class_weight
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

import warnings
warnings.filterwarnings("ignore")

### Pré-processando os dados

In [3]:
# coletando o caminho dos arquivos dos dados do hospital shenzen
filelist_shenzen = glob.glob('../0-datasets/montgomery-shenzen/arquivos-descompactados/ChinaSet_AllFiles/ChinaSet_AllFiles/CXR_png/*.png')

In [4]:
# quantidade de imagens disponíveis no dataset
print('quantidade de imagens:', str(len(filelist_shenzen)))

quantidade de imagens: 662


In [5]:
def extract_label(file_list):
    
    # inicializando uma lista vazia
    labels = []
    
    # iterando na lista de arquivos
    for file in tqdm(file_list):
        # detectando as classes presentes no nome da imagem
        current_label = re.findall('[0-9]{4}_(.+?).png', file)
        # adicionando a lista de rótulos as classes correspondentes a cada uma das imagens
        if current_label[0] == '0':
          labels.append('normal')
        else:
          labels.append('abnormal')
        
    return labels

In [6]:
# extraindo os rótulos
labels = extract_label(filelist_shenzen)

100%|█████████████████████████████████████████████████████████████████████████████| 662/662 [00:00<00:00, 75439.58it/s]


In [7]:
# visualizando a quantidade de rótulos
print('quantidade de rótulos:', str(len(labels)))

quantidade de rótulos: 662


In [8]:
# criando um dataframe com os caminhos das imagens
full_data = pd.DataFrame(filelist_shenzen, columns = ['filepath'])
# adicionando os rótulos em cada imagem
full_data['target'] = labels

In [9]:
# separando os dados de treinamento e de teste
train_df, test_df = train_test_split(full_data, stratify = full_data['target'],
                                     test_size = 0.2, random_state = 42)

In [10]:
# separando os dados de validação dos dados de treinamento
train_df, validation_df = train_test_split(train_df, stratify = train_df['target'],
                                           test_size = 0.2, random_state = 42)

In [11]:
# visualizando a quantidade de dados
print('quantidade de imagens de treinamento:', len(train_df['filepath']))
print('quantidade de rótulos de treinamento:', len(train_df['target']))
print('quantidade de imagens de teste:', len(test_df['filepath']))
print('quantidade de rótulos de teste:', len(test_df['target']))
print('quantidade de imagens de validação:', len(validation_df['filepath']))
print('quantidade de rótulos de validação:', len(validation_df['target']))

quantidade de imagens de treinamento: 423
quantidade de rótulos de treinamento: 423
quantidade de imagens de teste: 133
quantidade de rótulos de teste: 133
quantidade de imagens de validação: 106
quantidade de rótulos de validação: 106


In [12]:
# organizando um dicionário para realizar o balanceamento nos dados das classes
class_weights = class_weight.compute_class_weight('balanced', np.unique(train_df['target']),
                                                  train_df['target'])
class_weight = {0: class_weights[0], 1: class_weights[1]}

In [13]:
# salvando os dataframes manipulados em arquivos csv
train_df.to_csv('train_df.csv')
validation_df.to_csv('validation_df.csv')
test_df.to_csv('test_df.csv')

### Gerador de dados para a rede pelo Keras

In [14]:
# normalizando as imagens de treinamento e aplicando aumento de dados
image_generator = ImageDataGenerator(samplewise_center = True, samplewise_std_normalization = True,
                                     rotation_range = 20, zoom_range = 0.2)

# criando o gerador de imagens de treinamento 
train_generator = image_generator.flow_from_dataframe(
                                                      dataframe = train_df,
                                                      directory = '',
                                                      x_col = 'filepath',
                                                      y_col = 'target',
                                                      batch_size = 32,
                                                      seed = 42,
                                                      shuffle = True,
                                                      class_mode = 'categorical',
                                                      target_size = (256, 256))
# criando o gerador de imagens de validação 
valid_generator = image_generator.flow_from_dataframe(
                                                      dataframe = validation_df,
                                                      directory = '.', 
                                                      x_col = 'filepath',
                                                      y_col = 'target',
                                                      batch_size = 32,
                                                      seed = 42,
                                                      shuffle = True,
                                                      class_mode = 'categorical',
                                                      target_size = (256, 256))

# normalizando as imagens de teste 
test_datagen = ImageDataGenerator(samplewise_center = True, samplewise_std_normalization = True)

test_generator = test_datagen.flow_from_dataframe(
                                                  dataframe = test_df, 
                                                  directory = '.',
                                                  x_col = 'filepath',
                                                  y_col = 'target',
                                                  batch_size = 32,
                                                  seed = 42,
                                                  shuffle = True,
                                                  class_mode = 'categorical',
                                                  target_size = (256, 256))

Found 423 validated image filenames belonging to 2 classes.
Found 106 validated image filenames belonging to 2 classes.
Found 133 validated image filenames belonging to 2 classes.


In [15]:
# observando os índices das classes encontradas pelo gerador de dados do keras
train_generator.class_indices

{'abnormal': 0, 'normal': 1}